## Stok Prediction
Refer: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [2]:
path = "dataset/AAPL_data.csv"
df = pd.read_csv(path)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
date      1259 non-null object
open      1259 non-null float64
high      1259 non-null float64
low       1259 non-null float64
close     1259 non-null float64
volume    1259 non-null int64
Name      1259 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 69.0+ KB
None


In [3]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,67.7142,68.4014,66.8928,67.8542,158168416,AAPL
1,2013-02-11,68.0714,69.2771,67.6071,68.5614,129029425,AAPL
2,2013-02-12,68.5014,68.9114,66.8205,66.8428,151829363,AAPL
3,2013-02-13,66.7442,67.6628,66.1742,66.7156,118721995,AAPL
4,2013-02-14,66.3599,67.3771,66.2885,66.6556,88809154,AAPL


In [4]:
df['date'] = pd.to_datetime(df['date'])
df.set_axis(df['date'], inplace=True)
df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

In [5]:
df.head()

,date,close,Name
date,,,
2013-02-08,2013-02-08,67.8542,AAPL
2013-02-11,2013-02-11,68.5614,AAPL
2013-02-12,2013-02-12,66.8428,AAPL
2013-02-13,2013-02-13,66.7156,AAPL
2013-02-14,2013-02-14,66.6556,AAPL


## Data pre-processing

In [6]:
close_data = df['close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['date'][:split]
date_test = df['date'][split:]

print(len(close_train))
print(len(close_test))

1007
252


<b>TimeseriesGenerator</b> method generates <br/>
data from, [2, 3, 4, 5, 4, 6, 7, 6, 8, 9] to <br/>
[2, 3, 4] -> [5] <br/>
[3, 4, 5] -> [4] <br/>
[4, 5, 4] -> [6] <br/>
[5, 4, 6] -> [7] <br/>
[4, 6, 7] -> [6] <br/>
[6, 7, 6] -> [8] <br/>
[7, 6, 8] -> [9] <br/>
Because time series predict from past data format, If <b>look_back</b> is 3 then a single stock predicted from last look_back(3) values

In [7]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

## Model building and training

In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 150
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
50/50 [==============================] - 1s 29ms/step - loss: 97346.0181
Epoch 2/150
50/50 [==============================] - 1s 18ms/step - loss: 5883.8779
Epoch 3/150
50/50 [==============================] - 1s 18ms/step - loss: 251.2887
Epoch 4/150
50/50 [==============================] - 1s 18ms/step - loss: 216.7450
Epoch 5/150
50/50 [==============================] - 1s 18ms/step - loss: 193.8987
Epoch 6/150
50/50 [==============================] - 1s 18ms/step - loss: 174.2708: 0s - 
Epoch 7/150
50/50 [==============================] - 1s 18ms/step - loss: 120.1000
Epoch 8/150
50/50 [==============================] - 1s 17ms/step - loss: 185.1876
Epoch 9/150
50/50 [==============================] - 1s 19ms/step - loss: 189.4707
Epoch 10/150
50/50 [==============================] - 1s 19ms/step - loss: 138.4902
Epoch 11/150
50/50 [===================

50/50 [==============================] - 1s 18ms/step - loss: 8.6866
Epoch 92/150
50/50 [==============================] - 1s 18ms/step - loss: 12.1844
Epoch 93/150
50/50 [==============================] - 1s 18ms/step - loss: 8.1488
Epoch 94/150
50/50 [==============================] - 1s 18ms/step - loss: 8.1673
Epoch 95/150
50/50 [==============================] - 1s 20ms/step - loss: 15.4096
Epoch 96/150
50/50 [==============================] - 1s 17ms/step - loss: 9.6850
Epoch 97/150
50/50 [==============================] - 1s 16ms/step - loss: 9.7458
Epoch 98/150
50/50 [==============================] - 1s 19ms/step - loss: 8.7625
Epoch 99/150
50/50 [==============================] - ETA: 0s - loss: 10.05 - 1s 13ms/step - loss: 9.9252
Epoch 100/150
50/50 [==============================] - 1s 20ms/step - loss: 9.8819: 0s - loss: 8.320 - ETA: 0s - loss: 8
Epoch 101/150
50/50 [==============================] - 1s 17ms/step - loss: 8.0296
Epoch 102/150
50/50 [========================

A simple architecture of LSTM units trained using Adam optimizer and Mean Squared Loss function for 25 epochs. Note that instead of using model.fit(), we use model.fit_generator() because we have created a data generator.

## Prediction

In [10]:
import plotly.graph_objects as go
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()